In [22]:
#!/usr/bin/env python

import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from utils import dlc_practical_prologue as prologue
from utils.helpers import compute_nb_errors, train_model, update_target_type

from models.proj1.weightSharing import WeightSharing

import torch

from torch import optim
from torch.autograd import Variable

import matplotlib
import cv2
import random
import numpy as np

In [2]:
# load the data
train_input, train_target, train_classes, test_input, test_target, test_classes = \
    prologue.generate_pair_sets(nb=1000)

/home/mbanga/.pyenv/versions/deeplearning/lib/python3.7/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/mbanga/.pyenv/versions/deeplearning/lib/python3.7/site-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/mbanga/.pyenv/versions/deeplearning/lib/python3.7/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/mbanga/.pyenv/versions/deeplearning/lib/python3.7/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [3]:
def standard_wrapper(*data):
    return [Variable(t) for t in data]

In [4]:
def custom_wrapper(*data):
    return None

In [5]:
def wrap_data(*data, type_='standard'):
    switch = {
        'standard': standard_wrapper,
        'Net3': custom_wrapper
    }
    
    return switch.get(type_, None)(*data)    

In [6]:
def summary(model, optimizer, lr, training_time=None, NB_EPOCHS=None, MINI_BATCH_SIZE=None):
    print('model: {:>5}, criterion: {:>10}, optimizer: {:>10}, learning rate: {:6}, num epochs: {:3}, '
    'mini batch size: {:3}, training time: {:5.2f}, train error: {:5.2f}%, test error: {:5.2f}%'.format(
        model.__class__.__name__,
        model.criterion.__class__.__name__,
        opt.__class__.__name__,
        lr,
        NB_EPOCHS,
        MINI_BATCH_SIZE,
        training_time,
        compute_nb_errors(model, train_input, train_target, MINI_BATCH_SIZE) / train_input.size(0) * 100,
        compute_nb_errors(model, test_input, test_target, MINI_BATCH_SIZE) / test_input.size(0) * 100
        )
    )

In [117]:
# Compute mean and std for data
mean, std = train_input.mean(), train_input.std()

# Standardize data
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

# Wrap data in a Variable object
train_input, train_target, train_classes, test_input, test_target = \
    wrap_data(train_input, train_target, train_classes, test_input, test_target, type_='standard')


train_input_Net3 = train_input.view(-1, 1, 14, 14)
train_target_Net3 = train_classes.flatten()

# test different configurations
NB_EPOCHS = 25
MINI_BATCH_SIZE = 100
models = [WeightSharing]
opts = [torch.optim.SGD, torch.optim.Adam]
lrs = [1e-1, 1e-2, 1e-3]


for m in models:
    model = m()
    train_target, test_target = update_target_type(model, train_target, test_target)
    for opt in opts[:1]:
        for lr in lrs[:1]:      
            tt = train_model(model, opt(model.parameters(), lr=lr), NB_EPOCHS, \
                    train_input, train_target, MINI_BATCH_SIZE)
            
            summary(model, opt, lr, training_time=tt, NB_EPOCHS=NB_EPOCHS, MINI_BATCH_SIZE=MINI_BATCH_SIZE)                   

model: WeightSharing, criterion: BCEWithLogitsLoss, optimizer:       type, learning rate:    0.1, num epochs:  25, mini batch size: 100, training time:  4.31, train error: 11.30%, test error: 17.80%


In [8]:
optim = torch.optim.SGD(WeightSharing().parameters(), lr=0.001)

optim.param_groups[0]['lr']

0.001

In [37]:
import time
from datetime import datetime
import visdom
from math import ceil

In [43]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    random.seed(0)
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [25]:
def split_data(y, tx, ratio=0.2):
    assert len(y) == len(tx)
    assert 0 < ratio and ratio < 1
    
    # set seed
    random.seed(0)
    data_size = len(y)

    shuffle_indices = np.random.permutation(np.arange(data_size))
    shuffled_y = y[shuffle_indices]
    shuffled_tx = tx[shuffle_indices]
    
    cut = int(data_size*ratio)
    return shuffled_tx[cut:], shuffled_tx[:cut], shuffled_y[cut:], shuffled_y[:cut]

In [116]:
class Visualization():
    def __init__(self, env_name=None):
        if env_name is None:
            env_name = str(datetime.now().strftime("%d-%m %Hh%M"))
        self.env_name = env_name
        self.vis = visdom.Visdom(env=self.env_name)
        self.loss_win = None
        self.accuracy_win = None
        
    def plot_loss(self, loss, epoch, opt_name):
        self.loss_win = self.vis.line(
            [loss],
            [epoch],
            win=self.loss_win,
            update='append' if self.loss_win else None,
            opts=dict(
                xlabel='epoch',
                ylabel=opt_name,
                title='Loss (mean per 10 steps)',
            )
        )
        
    def plot_accuracy(self, train_accuracy, validation_accuracy, epoch, opt_name):
        self.accuracy_win = self.vis.line(
            [train_accuracy],
            [epoch],
            win=self.accuracy_win,
            update='append' if self.accuracy_win else None,
            name='train',
            opts=dict(
                xlabel='epoch',
                ylabel=opt_name,
                title='% Accuracy (mean per 10 steps)',
            )
        )
        
        self.accuracy_win = self.vis.line(
            [validation_accuracy],
            [epoch],
            win=self.accuracy_win,
            update='append' if self.accuracy_win else None,
            name='validation',
            opts=dict(
                xlabel='epoch',
                ylabel=opt_name,
                title='% Accuracy (mean per 10 steps)',
            )
        )

In [51]:
def compute_accuracy(output, target, mini_batch_size):
    assert len(output) == len(target) 
    # Define number of samples
    nb_samples = len(output)
    # Define number of batches
    num_batches = ceil(len(output) / mini_batch_size)
    
    # initialize number of errors
    nb_errors = 0
    for x, y in batch_iter(output, target, mini_batch_size, num_batches=num_batches, shuffle=False):
        for i in range(len(x)):
            if x[i] != y[i]:
                nb_errors += 1
    
    return 1 - nb_errors / len(output)

In [114]:
def train_model(model, optimizer, nb_epochs, data_input, data_target,mini_batch_size):
    train_input, validation_input, train_target, validation_target = split_data(data_target, data_input)
    
    n_samples = train_input.size(0)
    
    vis = Visualization()
    losses = []
    start = time.time()
    for e in range(0,nb_epochs):
        # Train the model during epoch e
        model.train()
        running_loss = 0.0
        for b in range(0, n_samples, mini_batch_size):
            # zero the parameter gradients
            model.zero_grad()
            # Forward pass
            output = model(train_input.narrow(0, b, mini_batch_size))
            # Get corresponding target values
            target = train_target.narrow(0, b, mini_batch_size)
            # Backward pass
            loss = model.criterion(output, target)
            loss.backward()                        
            # Update weights
            optimizer.step()
            # Update loss
            running_loss += loss.item()
        
        losses.append(running_loss)
        # Get predictions of the model
        preds = model.predict(train_input)
        # Get current training accuracy 
        accuracy = compute_accuracy(preds, train_target, mini_batch_size)
        

        # Get metrcis on validation data for epoch e
        model.eval()
        # Forward pass
        validation_preds = model.predict(validation_input)
        # 
        validation_output = model(validation_input)
        # Criteron
        validation_loss = model.criterion(validation_output, validation_target)
        # Get validation accuracy 
        validation_accuracy = compute_accuracy(validation_preds, validation_target, mini_batch_size)
        
        # Update accuracy plot
        vis.plot_accuracy(accuracy, validation_accuracy, e, optimizer.__class__.__name__)
        
        
        
        vis.plot_loss(running_loss, e, optimizer.__class__.__name__)
        end = time.time()

    training_time = end-start

    return training_time